In [1]:
import urllib3
import json
import time

In [2]:
http = urllib3.PoolManager()
url = 'http://192.168.0.152/api/vE1eSkyDYr/lights/'

<h3> Get Light Data </h3>

In [3]:
response = http.request('GET', url)
readableData = json.loads(response.data.decode('utf-8'))

lamp = 6
print ("\nData For Lamp Number " + str(lamp) + "\n")

on = readableData[str(lamp)]['state']['on']
bri = readableData[str(lamp)]['state']['bri']
reachable = readableData[str(lamp)]['state']['reachable']
name = readableData[str(lamp)]['name']

print ("Name : " + name + "\nOn : " + str(on) + "\nBrightness : " + str(bri) + "\nReachable : " + str(reachable) + "\n")


print ("All Data : \n")
readableData



Data For Lamp Number 6

Name : Zohair’s Ceiling Light
On : False
Brightness : 204
Reachable : True

All Data : 



{'10': {'capabilities': {'certified': True,
   'control': {'maxlumen': 840, 'mindimlevel': 5000},
   'streaming': {'proxy': False, 'renderer': False}},
  'config': {'archetype': 'classicbulb',
   'direction': 'omnidirectional',
   'function': 'functional'},
  'manufacturername': 'Philips',
  'modelid': 'LWB014',
  'name': 'Living Room Lamp',
  'productid': 'Philips-LWB014-1-A19DLv4',
  'productname': 'Hue white lamp',
  'state': {'alert': 'none',
   'bri': 203,
   'mode': 'homeautomation',
   'on': False,
   'reachable': True},
  'swconfigid': '321D79EA',
  'swupdate': {'lastinstall': '2018-01-18T09:02:19', 'state': 'noupdates'},
  'swversion': '1.29.0_r21169',
  'type': 'Dimmable light',
  'uniqueid': '00:17:88:01:02:a1:98:db-0b'},
 '5': {'capabilities': {'certified': True,
   'control': {'maxlumen': 840, 'mindimlevel': 5000},
   'streaming': {'proxy': False, 'renderer': False}},
  'config': {'archetype': 'classicbulb',
   'direction': 'omnidirectional',
   'function': 'functional'},


<h3> Set Ligts </h3>

In [4]:
lamp = 6
on = False
bri = 200
http.request('PUT', 'http://192.168.0.152/api/vE1eSkyDYr/lights/' + str(lamp) + '/state/', body = '{"on" : ' + str(on).lower() + "," + '"bri" : ' +  str(bri) + '}')

<h3> Work With Power Cuts </h3>

In [ ]:
# Change The Following Dictionaries To Reflect Your Lamp Numbers
previousBriDict = {9 : 100, 10 : 100, 6 : 100, 7 : 100, 8 : 100, 5 : 100}
previousStateDict = {9 : True, 10 : True, 6 : True, 7 : True, 8 : True, 5 : True}

while True:
    
    reachableLamps = []
    reachableCount = 0
    powerCutConfidence = 0
    
    # Incase Power Cut Takes Out WiFi Too
    errorFlag = True
    
    while errorFlag == True:
    
        try :
            response = http.request('GET', url)
            readableData = json.loads(response.data.decode('utf-8'))
            errorFlag = False

        except : 
            errorFlag = True
            time.sleep(5)
            
    for lamp in readableData:
        
        reachable = readableData[str(lamp)]['state']['reachable']
        
        if reachable == True:
            
            reachableLamps.append(lamp)
            reachableCount = reachableCount + 1
    
    for lamp in reachableLamps:
        
        if readableData[str(lamp)]['state']['bri'] == 254 and readableData[str(lamp)]['state']['on'] == True:
            
            powerCutConfidence = powerCutConfidence + 1

    if powerCutConfidence == reachableCount : 
        
        for lamp in reachableLamps:
            
            # Set To Previous Values
            http.request('PUT', 'http://192.168.0.152/api/vE1eSkyDYr/lights/' + str(lamp) + '/state/', body = '{"on" : ' + str(previousStateDict[lamp]).lower() + "," + '"bri" : ' +  str(previousBriDict[lamp]) + '}')
            
            # print (readableData[str(lamp)]['name'])
            # print ("Setting To Previous On : ", str(previousStateDict[lamp]))
            # print ("Setting To Previous Brightness : ", str(previousBriDict[lamp]))
            # print ("")
        
    else :
        
        for lamp in reachableLamps:
    
            # Set Current Values
            previousStateDict[lamp] = readableData[str(lamp)]['state']['on']
            previousBriDict[lamp] = readableData[str(lamp)]['state']['bri']


    time.sleep(10)
